# ORB-SLAM3 Stereo Inertial

In [ ]:
# Install dependencies
%pip install opencv-python matplotlib pandas requests jupyter_bokeh ipywidgets tqdm

In [2]:
import cv2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Iterator, Tuple, Optional
#import pandas as pd
try:
    import orb_slam3
except ImportError:
    from pathlib import Path
    sys.path.append(str(Path(os.getcwd()) / ".." / ".." / "lib"))
    import orb_slam3

In [3]:
from bokeh.io import output_notebook, show, export_svg
from bokeh.plotting import figure

output_notebook()    

Loading BokehJS ...

In [4]:
# Helper that displays an inline video
import matplotlib
import matplotlib.animation as animation
from IPython.display import HTML

animation.embed_limit = 500

def display_video(frames, framerate=30):
    h, w = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')
    fig, ax = plt.subplots(1, 1, figsize=(w / dpi, h / dpi), dpi=dpi)
    matplotlib.use(orig_backend)
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0], cmap='gray', vmin=0, vmax=255)
    def update(frame):
        im.set_data(frame)
        return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames, 
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

In [5]:
# VSLAM Dataset loader
class Dataset:
    def __init__(self, path: str, start: int = 0, end: int = -1) -> None:
        self._root = Path(path)
        if not self._root.exists():
            raise "%s does not exist" % path

        self._cam0 = self._root / 'cam0' / 'data'
        self._cam1 = self._root / 'cam1' / 'data'
        # self._imu0 = np.loadtxt(str(self._root / 'imu0' / 'data.csv'), delimiter=',', skiprows=1)
        self._is_stereo = self._cam1.exists() and self._cam1.is_dir()

        tsfile = self._root / 'timestamp_cam.txt'
        if not tsfile.exists():
            raise Exception("timestamp_cam.txt does not exist")

        self._timestamps = np.loadtxt(tsfile, dtype=int, delimiter=',')[start*20:end*20]
    
    def images(self) -> Iterator[Tuple[cv2.Mat, Optional[cv2.Mat]]]:
        for t in self._timestamps:
            img0 = cv2.imread(str(self._cam0 / ('%d.png' % t)), cv2.IMREAD_GRAYSCALE)
            if self._is_stereo:
                img1 = cv2.imread(str(self._cam1 / ('%d.png' % t)), cv2.IMREAD_GRAYSCALE)
                yield (img0, img1)
            else:
                yield (img0)
    
    def __iter__(self):
        for t in self._timestamps:
            img0 = cv2.imread(str(self._cam0 / ('%d.png' % t)), cv2.IMREAD_GRAYSCALE)
            if self._is_stereo:
                img1 = cv2.imread(str(self._cam1 / ('%d.png' % t)), cv2.IMREAD_GRAYSCALE)
                yield (t, img0, img1)
            else:
                yield (t, img0)

## Load dataset recorded in simulator scene

In [9]:
# Load sample dataset
#dataset = Dataset("dataset-20230126081954", start=5, end=20)
dataset = Dataset("dataset-unity")

In [1]:
# Display stereo camera frames
frames = list()
size = (400, 300)
for (imgl, imgr) in dataset.images()[0:100]:
    stereo_img = cv2.hconcat([cv2.resize(imgl, dsize=size), cv2.resize(imgr, dsize=size)])
    frames.append(stereo_img)
    
display_video(frames, framerate=15)    

NameError: name 'dataset' is not defined

## Initialize ORB-SLAM3

In [ ]:
# Initialize ORB-SLAM3 System
MONOCULAR = 0
STEREO = 1
RGBD = 2
IMU_MONO = 3
IMU_STEREO = 4
IMU_RGBD = 5

voc = str(Path("../../Vocabulary/ORBvoc.txt").resolve())
#cfg = str(Path("dataset-20230126081954/humblebee.yaml").resolve())
cfg = str(Path("dataset-unity/orb-slam3.yaml"))
slam = orb_slam3.System(voc, cfg, STEREO, False, 0)

In [24]:
# Run Stereo VSLAM
poses = list()
ts = list()
start = 0
for t, img0, img1 in dataset:
    if start == 0:
        start = t/1e9
    tcw = slam.track_stereo(img0, img1, t/1e9, np.empty(0))
    ts.append(t/1e9 - start)
    poses.append(tcw)

First KF:0; Map init KF:0
New Map created with 1 points
TRACK_REF_KF: Less than 15 matches!!
Fail to track local map!
LM: Active map reset recieved
LM: Active map reset, waiting...
LM: Reseting current map in Local Mapping...
LM: End reseting Local Mapping...
LM: Reset free the mutex
LM: Active map reset, Done!!!
mnFirstFrameId = 0
mnInitialFrameId = 0
1 Frames set to lost
First KF:1; Map init KF:0
New Map created with 0 points
TRACK_REF_KF: Less than 15 matches!!
Fail to track local map!
LM: Active map reset recieved
LM: Active map reset, waiting...
LM: Reseting current map in Local Mapping...
LM: End reseting Local Mapping...
LM: Reset free the mutex
LM: Active map reset, Done!!!
mnFirstFrameId = 2
mnInitialFrameId = 1
2 Frames set to lost
First KF:2; Map init KF:1
New Map created with 0 points
TRACK_REF_KF: Less than 15 matches!!
Fail to track local map!
LM: Active map reset recieved
LM: Active map reset, waiting...
LM: Reseting current map in Local Mapping...
LM: End reseting Local

In [25]:
# Plot local pose estimation
pose = np.array(poses)
p = figure(width=800, height=400, title="Vision Odometry Position")
p.line(ts, pose[:,1], line_width=2, line_color="red", legend_label="X")
p.line(ts, pose[:,2], line_width=2, line_color="green", legend_label="Y")
p.line(ts, pose[:,3], line_width=2, line_color="blue", legend_label="Z")
p.legend.location = "top_left"
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Pos'
p.output_backend = 'svg'

show(p)

In [26]:
p = figure(width=400, height=400)
p.circle(pose[:,0], pose[:,2] * -1, line_width=1)
p.xaxis.axis_label = 'X [m]'
p.yaxis.axis_label = 'Y [m]'
show(p)